In [1]:
from _params_models import rf_model_params
from _ml_models import *
from _plots import *
from _util import *

import io
import sys
import time
import pickle
import sympy as sp
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from typing import Tuple
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

In [2]:
# carregando dados
sleep1_dash = pd.read_csv("assets/data/part2/sleep_1/data_dash.log", delimiter=",", header=None, skiprows=1, on_bad_lines='skip')
sleep10_dash = pd.read_csv("assets/data/part2/sleep_10/data_dash.log", delimiter=",", header=None, skiprows=1, on_bad_lines='skip')
sleep05_dash = pd.read_csv("assets/data/part2/sleep_05/data_dash.log", delimiter=",", header=None, skiprows=1, on_bad_lines='skip')
sleep0_dash = pd.read_csv("assets/data/part2/sleep_0/data_dash.log", delimiter=",", header=None, skiprows=1, on_bad_lines='skip')

sleep1_int = pd.read_csv("assets/data/part2/sleep_1/data_int_.csv", delimiter=",", header=None, skiprows=1, on_bad_lines='skip')
sleep10_int = pd.read_csv("assets/data/part2/sleep_10/data_int_.csv", delimiter=",", header=None, skiprows=1, on_bad_lines='skip')
sleep05_int = pd.read_csv("assets/data/part2/sleep_05/data_int_.csv", delimiter=",",  header=None, skiprows=1, on_bad_lines='skip')
sleep0_int = pd.read_csv("assets/data/part2/sleep_0/data_int_.csv", delimiter=",",  header=None, skiprows=1, on_bad_lines='skip')

sleep1_int = remove_useless_attribute(sleep1_int).reset_index(drop=True)
sleep10_int = remove_useless_attribute(sleep10_int).reset_index(drop=True)
sleep05_int = remove_useless_attribute(sleep05_int).reset_index(drop=True)
sleep0_int = remove_useless_attribute(sleep0_int).reset_index(drop=True)

columns_name_int = [
    "timestamp",
    "ingress_global_timestamp3",
    "egress_global_timestamp3",
    "enq_timestamp3",
    "enq_qdepth3",
    "deq_timedelta3",
    "deq_qdepth3",
    "ingress_global_timestamp2",
    "egress_global_timestamp2",
    "enq_timestamp2",
    "enq_qdepth2",
    "deq_timedelta2",
    "deq_qdepth2",
    "ingress_global_timestamp1",
    "egress_global_timestamp1",
    "enq_timestamp1",
    "enq_qdepth1",
    "deq_timedelta1",
    "deq_qdepth1",
]
columns_name_dash = [
    "timestamp",
    "inputBitrate",
    "demuxBitrate",
    "demuxCorrupted",
    "demuxDiscontinuity",
    "sendBitrate",
    "framesDisplayed",
    "playedAudioBuffers",
    "decodedVideo",
    "decodedAudio",
    "framesDisplayedAux",
    "playedAudioBuffersAux",
    "decodedVideoAux",
    "decodedAudioAux",
    "framesDisplayedCalc",
    "playedAudioBuffersCalc",
    "decodedVideoCalc",
    "decodedAudioCalc",
]

sleep1_int.columns = columns_name_int
sleep10_int.columns = columns_name_int
sleep05_int.columns = columns_name_int
sleep0_int.columns = columns_name_int

sleep1_dash.columns = columns_name_dash
sleep10_dash.columns = columns_name_dash
sleep05_dash.columns = columns_name_dash
sleep0_dash.columns = columns_name_dash

In [3]:
sleep1_features, sleep1_labels = merge_dataset(sleep1_int, sleep1_dash)
sleep10_features, sleep10_labels = merge_dataset(sleep10_int, sleep10_dash)
sleep05_features, sleep05_labels = merge_dataset(sleep05_int, sleep05_dash)
sleep0_features, sleep0_labels = merge_dataset(sleep0_int, sleep0_dash)

In [4]:
def evaluate_model(features, labels, best_params, sleep_duration):
    start_time = time.time()
    mae, nmae, model = default_random_forest_model(features, labels, best_params)
    end_time = time.time()

    elapsed_time = end_time - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    milliseconds = int((elapsed_time % 1) * 1000)
    print(f"Sleep {sleep_duration} -> MAE = {mae:.5f}, NMAE = {nmae * 100:.2f}%, Tempo de treino: {minutes} minutos/{seconds} segundos/ {milliseconds} milissegundos")
    return mae, nmae, model

In [ ]:
best_params = rf_model_params(
    n_estimators=90,
    min_samples_split=5,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=44,
    bootstrap=True,
    n_splits=5,
    random_state=42,
    shuffle=False,
    test_size=0.2,
    verbose = 0,
    n_jobs=2
)

mae_sleep0, nmae_sleep0, model_sleep0 = evaluate_model(sleep0_features, sleep0_labels, best_params, 0)
mae_sleep05, nmae_sleep05, model_sleep05 = evaluate_model(sleep05_features, sleep05_labels, best_params, 0.5)
mae_sleep1, nmae_sleep1, model_sleep1 = evaluate_model(sleep1_features, sleep1_labels, best_params, 1)
mae_sleep10, nmae_sleep10, model_sleep10 = evaluate_model(sleep10_features, sleep10_labels, best_params, 10)

In [ ]:
def evaluate_and_plot_combined(models, features, labels, start_idx, end_idx, titles, dataset_test):
    predictions = [model.predict(features) for model in models]
    
    nmae_values = [nmae(labels, pred) for pred in predictions]
    for title, nmae_value in zip(titles, nmae_values):
        print(f"{title} - NMAE: {nmae_value*100:.2f}%")
    
    fig, axs = plt.subplots(2, 2, figsize=(16, 10), constrained_layout=True)
    t = np.arange(start_idx, end_idx)

    for i, (ax, prediction, title) in enumerate(zip(axs.flatten(), predictions, titles)):
        ax.plot(t, labels[start_idx:end_idx], label='Labels', color='orange')
        ax.plot(t, prediction[start_idx:end_idx], label='Predictions', color='blue')
        ax.set_title(f'{title}: Labels vs Predictions Usando o modelo {dataset_test}')
        ax.set_xlabel('Índice')
        ax.set_ylabel('Valor')
        ax.grid(True)
        ax.legend()

    plt.show()

models = [model_sleep05, model_sleep1, model_sleep10, model_sleep0]
titles = ["Model Sleep0.5", "Model Sleep1", "Model Sleep10", "Model No Sleep"]

start_graph = 10
end_graph = 21500

evaluate_and_plot_combined(models, sleep05_features, sleep05_labels, start_graph, end_graph, titles,'dataset 0.5')


In [ ]:
def show_fps_subplots(data_dict, rows=2, cols=2):
    fig, axs = plt.subplots(rows, cols, figsize=(14, 10))
    axs = axs.flatten()
    for ax, (label, data) in zip(axs, data_dict.items()):
        ax.plot(data, label="FPS", color="blue", alpha=0.7)
        ax.set_title(f"FPS - {label}")
        ax.set_xlabel("Tempo")
        ax.set_ylabel("FPS")
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.show()


sleep1_dash = pd.read_csv(
    "assets/data/part2/sleep_1/data_dash.log",
    delimiter=",",
    header=None,
    skiprows=1,
    on_bad_lines="skip",
)
sleep10_dash = pd.read_csv(
    "assets/data/part2/sleep_10/data_dash.log",
    delimiter=",",
    header=None,
    skiprows=1,
    on_bad_lines="skip",
)
sleep05_dash = pd.read_csv(
    "assets/data/part2/sleep_05/data_dash.log",
    delimiter=",",
    header=None,
    skiprows=1,
    on_bad_lines="skip",
)
sleep0_dash = pd.read_csv(
    "assets/data/part2/sleep_0/data_dash.log",
    delimiter=",",
    header=None,
    skiprows=1,
    on_bad_lines="skip",
)
# dar nomes aos bois
columns_name_dash = [
    "timestamp",
    "inputBitrate",
    "demuxBitrate",
    "demuxCorrupted",
    "demuxDiscontinuity",
    "sendBitrate",
    "framesDisplayed",
    "playedAudioBuffers",
    "decodedVideo",
    "decodedAudio",
    "framesDisplayedAux",
    "playedAudioBuffersAux",
    "decodedVideoAux",
    "decodedAudioAux",
    "framesDisplayedCalc",
    "playedAudioBuffersCalc",
    "decodedVideoCalc",
    "decodedAudioCalc",
]
sleep1_dash.columns = columns_name_dash
sleep10_dash.columns = columns_name_dash
sleep05_dash.columns = columns_name_dash
sleep0_dash.columns = columns_name_dash
fps_data = {
    "Sleep 0": sleep0_dash["framesDisplayedCalc"],
    "Sleep 0.5": sleep05_dash["framesDisplayedCalc"],
    "Sleep 1": sleep1_dash["framesDisplayedCalc"],
    "Sleep 10": sleep10_dash["framesDisplayedCalc"],
}
show_fps_subplots(fps_data)

In [6]:
nmae_rf_1, best_rf_model_1, best_params_1 = default_rf_model_randomsearch(sleep1_features, sleep1_labels, 1000)
print(f"NMAE: {nmae_rf_1}%")
print(best_params_1)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
[CV 1/5] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=150;, score=-0.108 total time=   1.0s
[CV 2/5] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=150;, score=-0.107 total time=   1.1s
[CV 3/5] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=150;, score=-0.102 total time=   0.9s
[CV 4/5] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=150;, score=-0.111 total time=   0.9s
[CV 1/5] END bootstrap=True, max_depth=82, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=70;, score=-0.106 total time=   0.4s
[CV 2/5] END bootstrap=True, max_depth=82, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=70;, score=-0.107 total time=   0.4s

In [ ]:


# sleep 0.5
#NMAE: 0.10674233156158358%
#{'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 28, 'bootstrap': False}

# sleep 1
#NMAE: 0.1147273422449534%
#{'n_estimators': 90, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 55, 'bootstrap': True}

#sleep 10
#NMAE: 0.010394291682413846%
#{'n_estimators': 150, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 28, 'bootstrap': True}


#no sleep
#NMAE = 0.09908740836861213
##{'n_estimators': 90, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 55, 'bootstrap': True}

#NMAE: 0.09805375408158114%
#{'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 28, 'bootstrap': False}

In [9]:
best_params = rf_model_params(
    n_estimators=80,
    min_samples_split=5,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=28,
    bootstrap=True,
    n_splits=5,
    random_state=42,
    shuffle=False,
    test_size=0.2,
    verbose = 0,
    n_jobs=2
)
mae_sleep0, nmae_sleep0, model_sleep0 = evaluate_model(sleep0_features, sleep0_labels, best_params, 0)

Sleep 0 -> MAE = 2.84841, NMAE = 9.81%, Tempo de treino: 6 minutos/45 segundos/ 835 milissegundos


In [10]:
nmae_sleep0

np.float64(0.09805375408158114)